In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet101


class CBAM(nn.Module):
    def __init__(self, channels, reduction_ratio=16):
        super(CBAM, self).__init__()
        self.channels = channels
        self.reduction_ratio = reduction_ratio

        # Spatial attention layers
        self.spatial_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.spatial_max_pool = nn.AdaptiveMaxPool2d(1)
        self.spatial_fc1 = nn.Conv2d(channels, channels // reduction_ratio, kernel_size=1, stride=1, padding=0)
        self.spatial_relu = nn.ReLU(inplace=True)
        self.spatial_fc2 = nn.Conv2d(channels // reduction_ratio, channels, kernel_size=1, stride=1, padding=0)
        self.spatial_sigmoid = nn.Sigmoid()

        # Channel attention layers
        self.channel_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.channel_max_pool = nn.AdaptiveMaxPool2d(1)
        self.channel_fc1 = nn.Linear(channels, channels // reduction_ratio)
        self.channel_relu = nn.ReLU(inplace=True)
        self.channel_fc2 = nn.Linear(channels // reduction_ratio, channels)
        self.channel_sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Spatial attention
        spatial_avg = self.spatial_avg_pool(x)
        spatial_max = self.spatial_max_pool(x)
        spatial_avg = self.spatial_fc1(spatial_avg)
        spatial_max = self.spatial_fc1(spatial_max)
        spatial_attention = self.spatial_relu(spatial_avg + spatial_max)
        spatial_attention = self.spatial_fc2(spatial_attention)
        spatial_attention = self.spatial_sigmoid(spatial_attention)

        # Channel attention
        channel_avg = torch.flatten(self.channel_avg_pool(x), start_dim=1)
        channel_max = torch.flatten(self.channel_max_pool(x), start_dim=1)
        channel_avg = self.channel_fc1(channel_avg)
        channel_max = self.channel_fc1(channel_max)
        channel_attention = self.channel_relu(channel_avg + channel_max)
        channel_attention = self.channel_fc2(channel_attention)
        channel_attention = self.channel_sigmoid(channel_attention)
        channel_attention = channel_attention.unsqueeze(2).unsqueeze(3)

        # Apply attention
        x = x * spatial_attention + x * channel_attention

        return x


class ResNet101CBAM(nn.Module):
    def __init__(self, num_classes):
        super(ResNet101CBAM, self).__init__()
        self.resnet = resnet101(pretrained=True)
        self.cbam = CBAM(channels=2048)
        self.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)

        x = self.cbam(x)

        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x


# Create an instance of ResNet101CBAM model
model=ResNet101CBAM(10)
print(model)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 242MB/s]


ResNet101CBAM(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
     

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import ToTensor
from tqdm import tqdm
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
import os
from torch.utils.data import DataLoader

from google.colab import drive
drive.mount('/content/drive')
root_folder = 'drive/MyDrive/'
dataset_folder = os.path.join(root_folder, 'TEST_DATA')
# Create an instance of the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
dataset = ImageFolder(dataset_folder, transform=transform)
# Calculate the indices to split the dataset
dataset_size = len(dataset)
train_size = int(0.7 * dataset_size)
val_size = dataset_size - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders for training and validation
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)


# Create an instance of ResNet101CBAM model
num_classes = 3  # Change this to the appropriate number of classes for your dataset
model = ResNet101CBAM(num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 10
total_steps = len(train_dataloader)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch+1}/{num_epochs}"):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print average loss for the epoch
    epoch_loss = running_loss / total_steps
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

print("Training finished.")

# Save the trained model
torch.save(model.state_dict(), "resnet101_cbam.pth")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Epoch 1/10: 100%|██████████| 264/264 [10:58<00:00,  2.49s/it]


Epoch 1/10, Loss: 0.2895


Epoch 2/10: 100%|██████████| 264/264 [01:21<00:00,  3.25it/s]


Epoch 2/10, Loss: 0.0920


...

Epoch 8/10, Loss: 0.0080


Epoch 9/10: 100%|██████████| 264/264 [01:21<00:00,  3.25it/s]


Epoch 9/10, Loss: 0.0085


Epoch 10/10: 100%|██████████| 264/264 [01:21<00:00,  3.25it/s]


Epoch 10/10, Loss: 0.0092
Training finished.


In [ ]:


# Evaluation loop
model.eval()
total_correct = 0
total_samples = 0
test_loss=0

with torch.no_grad():
    for images, labels in tqdm(val_dataloader, desc="Validation"):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, dim=1)

        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
test_loss /= len(val_dataloader)
print(f"Accuracy on test data: {accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")


Validation: 100%|██████████| 114/114 [03:55<00:00,  2.07s/it]

Accuracy on test data: 98.67%
Test Loss: 0.0490


In [ ]:
import random

# Assuming you have a validation dataset
val_dataset_size = int(len(dataset) * 0.2)  # 20% of the original dataset
val_indices = random.sample(range(len(dataset)), val_dataset_size)
train_indices = [i for i in range(len(dataset)) if i not in val_indices]

train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Training loop
# ... (same as before)

# Evaluation loop
model.eval()
total_correct = 0
total_samples = 0
test_loss=0

with torch.no_grad():
    for images, labels in tqdm(val_dataloader, desc="Validation"):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, dim=1)

        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print(f"Accuracy on validation dataset: {accuracy * 100:.2f}%")
test_loss /= len(val_dataloader)
print(f"Validation Loss: {test_loss:.4f}")

Validation: 100%|██████████| 76/76 [00:10<00:00,  7.56it/s]

Accuracy on validation dataset: 99.17%
Validation Loss: 0.0224


In [ ]:
from sklearn.metrics import f1_score

true_labels = []
predicted_labels = []
# Evaluation loop
model.eval()

with torch.no_grad():
    for images, labels in tqdm(val_dataloader, desc="Validation"):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, dim=1)

        true_labels += labels.tolist()
        predicted_labels += predicted.tolist()
f1 = f1_score(true_labels, predicted_labels, average='macro')
print(f"F1 Score: {f1:.4f}")



Validation: 100%|██████████| 76/76 [00:10<00:00,  7.58it/s]

F1 Score: 0.9916
